# Chain of Density

テキストを要約する精度を高めるためのプロンプトです。
- 論文:https://arxiv.org/abs/2309.04269

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/nlp/summary(japanese)/chain_of_density.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from langchain.llms import OpenAI

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

llm_model = OpenAI(model_name= 'gpt-3.5-turbo', temperature=0)


/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### 通常のプロンプト

In [ ]:
text = """
量子コンピュータは問題を解く方法の違いにより、量子ゲート方式と量子アニーリング方式の大きく2つに分類されます。 量子ゲート方式は、量子状態にある素子の振る舞いや組み合わせで計算回路を作り、問題を解いていきます。超電導やイオントラップ、トポロジカルなど様々な実現手法が提案されています。従来型のコンピュータの上位互換として期待が高く、グーグルやIBMなどの大手ITベンダー、またリゲッティ・コンピューティングやIonQなどのスタートアップがハードウェアの開発を進めています。 量子アニーリング方式は、組み合わせ最適化問題を解くことくに特化しています。高温にした金属をゆっくり冷やすと構造が安定する「焼きなまし」の手法を応用して問題の解を求めていきます。商用化で先行するD-Wave Systemsのハードウェア（以下、D-Waveマシン）では、格子状に並べられた素子に相互作用を設定し、横磁場という信号をかけて、素子全体のエネルギーが最も低くなる状態を探し出していきます。日本ではNECが2023年までの実用化を発表しています。
"""

In [5]:
prompt = """
{text_}
------
上記を要約しください．
""".format(text_ = text)

In [6]:
output = llm_model(prompt)
print(output)

量子コンピュータは、量子ゲート方式と量子アニーリング方式の2つに分類されます。量子ゲート方式は、量子状態の素子の振る舞いや組み合わせを利用して計算回路を作り、問題を解く方法です。一方、量子アニーリング方式は、組み合わせ最適化問題を解くことに特化しており、金属を冷やす手法を応用して問題の解を求めます。量子コンピュータの商用化では、グーグルやIBMなどの大手ITベンダーやリゲッティ・コンピューティングやIonQなどのスタートアップがハードウェアの開発を進めています。また、D-Wave Systemsのハードウェアでは、格子状に並べられた素子に相互作用を設定し、最も低いエネルギー状態を探し出す方法を用いています。日本のNECも2023年までに実用化を目指しています。


### Chain of Density

In [2]:
text = """
量子コンピュータは問題を解く方法の違いにより、量子ゲート方式と量子アニーリング方式の大きく2つに分類されます。 量子ゲート方式は、量子状態にある素子の振る舞いや組み合わせで計算回路を作り、問題を解いていきます。超電導やイオントラップ、トポロジカルなど様々な実現手法が提案されています。従来型のコンピュータの上位互換として期待が高く、グーグルやIBMなどの大手ITベンダー、またリゲッティ・コンピューティングやIonQなどのスタートアップがハードウェアの開発を進めています。 量子アニーリング方式は、組み合わせ最適化問題を解くことくに特化しています。高温にした金属をゆっくり冷やすと構造が安定する「焼きなまし」の手法を応用して問題の解を求めていきます。商用化で先行するD-Wave Systemsのハードウェア（以下、D-Waveマシン）では、格子状に並べられた素子に相互作用を設定し、横磁場という信号をかけて、素子全体のエネルギーが最も低くなる状態を探し出していきます。日本ではNECが2023年までの実用化を発表しています。
"""

In [3]:
prompt = """
記事：{text_}

あなたは私に記事を依頼する。そして、その記事の簡潔で実体の濃い要約をどんどん作成する。

以下の2つのステップを5回繰り返す。

ステップ1. 先に生成された要約に欠けている、記事中の1～3個の有益なエンティティ（"; "区切り）を特定する。
ステップ2. 同じ長さで、前の要約のすべてのエンティティと詳細をカバーし、さらに欠落しているエンティティを加えた、より密度の高い新しい要約を書く。

欠けているエンティティとは
- メインストーリーに関連している、
- 具体的でありながら簡潔であること（5語以下）、
- 新規性（前回の要約にはない）、
- 忠実（記事中に存在する）、
- どこにでもある（記事のどこにでもある）。

ガイドライン

- 最初の要約は長く（4～5文、～80語）、しかし非常に非特異的で、欠落しているとマークされたエンティティ以上の情報をほとんど含まないこと。80ワードに達するには、過度に冗長な表現とフィラー（例：「この記事は論じている」）を使用する。
- 一語一語を大切にする：フローを改善し、エンティティを追加するスペースを作るために、前回の要約を書き直す。
- 融合、圧縮、「この記事は論じている」のような情報量の少ないフレーズの削除でスペースを作る。
- 要約は、高密度で簡潔でありながら自己完結的、つまり、記事なしでも容易に理解できるものにする。
- 欠落しているエンティティは新しい要約のどこに出現してもよい。
- 前の要約から実体を削除してはならない。スペースが確保できない場合は、新しいエンティティを少なくする。

各要約に全く同じ語数を使用することを忘れないこと。
JSONで答えなさい。JSONは、"Missing_Entities "と "Denser_Summary "をキーとする辞書のリスト（長さ5）でなければならない。
""".format(text_ = text)

In [4]:
output = llm_model(prompt)
print(output)

{"Missing_Entities": ["量子ゲート方式", "量子アニーリング方式"], "Denser_Summary": "量子コンピュータは量子ゲート方式と量子アニーリング方式に分類されます。量子ゲート方式は問題を解くための計算回路を作ります。量子アニーリング方式は組み合わせ最適化問題を解くことに特化しています。"}
